### Mod 4 Project - Starter Notebook

This notebook has been provided to you so that you can make use of the following starter code to help with the trickier parts of preprocessing the Zillow dataset. 

The notebook contains a rough outline the general order you'll likely want to take in this project. You'll notice that most of the areas are left blank. This is so that it's more obvious exactly when you should make use of the starter code provided for preprocessing. 

**_NOTE:_** The number of empty cells are not meant to infer how much or how little code should be involved in any given step--we've just provided a few for your convenience. Add, delete, and change things around in this notebook as needed!

# Some Notes Before Starting

This project will be one of the more challenging projects you complete in this program. This is because working with Time Series data is a bit different than working with regular datasets. In order to make this a bit less frustrating and help you understand what you need to do (and when you need to do it), we'll quickly review the dataset formats that you'll encounter in this project. 

## Wide Format vs Long Format

If you take a look at the format of the data in `zillow_data.csv`, you'll notice that the actual Time Series values are stored as separate columns. Here's a sample: 

<img src='~/../images/df_head.png'>

You'll notice that the first seven columns look like any other dataset you're used to working with. However, column 8 refers to the median housing sales values for April 1996, column 9 for May 1996, and so on. This This is called **_Wide Format_**, and it makes the dataframe intuitive and easy to read. However, there are problems with this format when it comes to actually learning from the data, because the data only makes sense if you know the name of the column that the data can be found it. Since column names are metadata, our algorithms will miss out on what dates each value is for. This means that before we pass this data to our ARIMA model, we'll need to reshape our dataset to **_Long Format_**. Reshaped into long format, the dataframe above would now look like:

<img src='~/../images/melted1.png'>

There are now many more rows in this dataset--one for each unique time and zipcode combination in the data! Once our dataset is in this format, we'll be able to train an ARIMA model on it. The method used to convert from Wide to Long is `pd.melt()`, and it is common to refer to our dataset as 'melted' after the transition to denote that it is in long format. 

# Helper Functions Provided

Melting a dataset can be tricky if you've never done it before, so you'll see that we have provided a sample function, `melt_data()`, to help you with this step below. Also provided is:

* `get_datetimes()`, a function to deal with converting the column values for datetimes as a pandas series of datetime objects
* Some good parameters for matplotlib to help make your visualizations more readable. 

Good luck!


# Step 1: Load the Data/Filtering for Chosen Zipcodes

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as plt
%matplotlib inline
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
import itertools

import warnings
def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()
    
df = pd.read_csv("zillow_data.csv")
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


In [2]:
import helper_functions as hf

In [3]:
from importlib import reload
from helper_functions import hf_ml as hfm
from helper_functions import hf_arima as hfa

In [4]:
from helper_functions import hf_arima as hfa

In [5]:
reload(hf)

<module 'helper_functions' from '/Users/bryanjamieson/flatiron/dsc-mod-4-project-online-ds-ft-090919/helper_functions/__init__.py'>

In [6]:
hfa.summ(3, 5)

8

In [7]:
# Created new df called df_GA containing only data from counties in Georgia
#region name = zipcode

df_GA = df.loc[df["State"] == 'GA']
df_GA.head(15)

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
28,71067,30349,Riverdale,GA,Atlanta,Clayton,29,90900.0,91100.0,91200.0,...,118700,119900,121400,122900,123800,124600,126200,127300,127700,128100
42,70829,30044,Lawrenceville,GA,Atlanta,Gwinnett,43,111400.0,111800.0,112200.0,...,169500,170900,172800,174700,176400,178500,180100,181000,182200,183400
51,70828,30043,Lawrenceville,GA,Atlanta,Gwinnett,52,135200.0,135500.0,135800.0,...,213600,216000,218400,220300,221900,223700,225500,226700,227800,228800
65,70874,30096,Duluth,GA,Atlanta,Gwinnett,66,119700.0,120000.0,120300.0,...,205900,207600,209400,211200,212700,214600,216500,218100,219800,221100
78,70810,30024,Suwanee,GA,Atlanta,Gwinnett,79,179600.0,181000.0,182400.0,...,330100,331700,333400,334800,336100,338600,341300,343200,344900,346300
101,71005,30281,Stockbridge,GA,Atlanta,Henry,102,94300.0,95700.0,97100.0,...,145100,146400,147400,148200,149300,149900,149400,149500,150800,151700
176,70825,30040,Cumming,GA,Atlanta,Forsyth,177,145200.0,145400.0,145900.0,...,277200,278300,279900,281600,283400,285900,288100,290000,292100,293200
177,70953,30188,Woodstock,GA,Atlanta,Cherokee,178,119200.0,119500.0,119800.0,...,213900,215100,216400,218200,220700,223200,225200,227100,229200,230800
213,70859,30080,Smyrna,GA,Atlanta,Cobb,214,133000.0,133100.0,133100.0,...,282400,284400,286900,290000,293700,297100,299400,302400,307100,311000
221,70842,30062,Marietta,GA,Atlanta,Cobb,222,150000.0,150500.0,150900.0,...,309200,310600,313300,316500,319100,321400,323100,324700,327000,328700


In [8]:
df_GA['CountyName'].unique()

array(['Clayton', 'Gwinnett', 'Henry', 'Forsyth', 'Cherokee', 'Cobb',
       'Fulton', 'Douglas', 'Chatham', 'Muscogee', 'Houston', 'Coweta',
       'Dekalb', 'Columbia', 'Clarke', 'Newton', 'Whitfield', 'Baldwin',
       'Hall', 'Catoosa', 'Bartow', 'Barrow', 'Floyd', 'Bulloch', 'Bibb',
       'Gordon', 'Fayette', 'Carroll', 'Lowndes', 'Murray', 'Walker',
       'Dougherty', 'Troup', 'Rockdale', 'Glynn', 'Dawson', 'Laurens',
       'Bryan', 'Tift', 'Lumpkin', 'Spalding', 'Pickens', 'Upson',
       'Walton', 'Lee', 'White', 'Thomas', 'Putnam', 'Stephens', 'Union',
       'Polk', 'Jackson', 'Butts', 'Gilmer', 'Effingham', 'Colquitt',
       'Camden', 'Peach', 'Ware', 'Hart', 'Greene', 'Fannin', 'Monroe',
       'Coffee', 'Habersham', 'Jones', 'Haralson', 'Morgan', 'Lamar',
       'Rabun', 'Liberty', 'Oconee', 'Madison', 'Harris', 'Wayne', 'Pike',
       'Banks', 'Wilcox', 'Calhoun'], dtype=object)

In [9]:
# After taking a look at the list of counties in Georgia, I picked 3 counties that I think real estate prices will 
# increase a lot in price over the next 20 years.

In [10]:
# New df "GA_Counties" contains real estate data only from the 3 counties I am looking into.

GA_Counties = ["Fulton", "Dekalb", "Cobb"]

In [11]:
df_GA_2 = df_GA.loc[(df_GA["CountyName"] == "Fulton")|
          (df_GA["CountyName"] == "Dekalb")|
          (df_GA["CountyName"] == "Cobb")]

In [12]:
# Here we are looking at the city names within our 3 counties
# buckhead,roswell,atlanta
df_GA_2["City"].unique()

array(['Smyrna', 'Marietta', 'Atlanta', 'Johns Creek', 'Powder Springs',
       'Milton', 'Lithonia', 'Acworth', 'Roswell', 'Kennesaw',
       'Brookhaven', 'Candler-Mcafee', 'Panthersville', 'Vinings',
       'Sandy Springs', 'Mableton', 'Tucker', 'Dunwoody', 'Fairburn',
       'Conley', 'North Decatur', 'Stone Mountain', 'North Druid Hills',
       'Chamblee', 'Decatur', 'Alpharetta', 'Austell', 'Union City',
       'Palmetto', 'Avondale Estates', 'Scottdale'], dtype=object)

In [13]:
#df_GA_2[df.loc["Fulton"]]
df_GA_2.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
213,70859,30080,Smyrna,GA,Atlanta,Cobb,214,133000.0,133100.0,133100.0,...,282400,284400,286900,290000,293700,297100,299400,302400,307100,311000
221,70842,30062,Marietta,GA,Atlanta,Cobb,222,150000.0,150500.0,150900.0,...,309200,310600,313300,316500,319100,321400,323100,324700,327000,328700
227,71050,30331,Atlanta,GA,Atlanta,Fulton,228,97100.0,97700.0,98400.0,...,130300,131300,132700,134000,135100,137000,139400,140800,141500,141600
254,70808,30022,Johns Creek,GA,Atlanta,Fulton,255,206900.0,207300.0,207800.0,...,393700,393900,395400,396600,397600,399600,402400,405100,407800,409000
296,70904,30127,Powder Springs,GA,Atlanta,Cobb,297,119300.0,119300.0,119300.0,...,182600,183600,185300,187300,189000,190600,192100,194000,197100,199500


In [14]:
# Zip Code Selection - Filter Zip codes by company preferences
# The real estate investment firm wants to know the top 5 zipcodes to invest in.
# Criteria:

# 1) Urbanization - zip code should be in top 50% according to sizerank variable.
#In the greater Atlanta area, commuting is tough with traffic, so living in a suburb or area within 45min driving distance is essential

# 2) Average House price
#House price should be close to the average house price for that zipcode.

# 3) ROI (Return on investment)

# 4) Risk
# Risk adverse investment company. Want zip codes to be selected according to a coefficient of variation below 70th percentile

In [15]:
print(df_GA_2.SizeRank.describe(),'\n')

count       59.000000
mean      2861.525424
std       2731.942421
min        214.000000
25%       1058.000000
50%       1927.000000
75%       3606.000000
max      11325.000000
Name: SizeRank, dtype: float64 



In [16]:
GA_sr_50 = df_GA_2.SizeRank.quantile(q=0.50)
print(f'GA Size Rank 50% cutoff value: {GA_sr_50}')

GA Size Rank 50% cutoff value: 1927.0


In [17]:
GA_zc_t50 = df_GA_2[df_GA_2['SizeRank']<GA_sr_50].drop(['RegionID','City','State','Metro','CountyName','SizeRank'],axis=1)
print(f'Amount of zipcodes: {len(GA_zc_t50)}')

Amount of zipcodes: 29


In [18]:
GA_zc_t50['yr_avg']=GA_zc_t50.iloc[:,-12:].mean(skipna=True, axis=1)

In [19]:
print(GA_zc_t50['yr_avg'].describe(),'\n')

count        29.000000
mean     333658.045977
std      195860.519484
min      116466.666667
25%      188908.333333
50%      277141.666667
75%      412816.666667
max      874691.666667
Name: yr_avg, dtype: float64 



In [20]:
q_60 = GA_zc_t50['yr_avg'].quantile(q=0.60)
print(f'Average 60% Cutoff Value: {round(q_60,2)}')

Average 60% Cutoff Value: 337753.33


In [21]:
q_35 = GA_zc_t50['yr_avg'].quantile(q=0.35)
print(f'Average 35% Cutoff Value: {round(q_35,2)}')

Average 35% Cutoff Value: 219516.67


In [22]:
# Get data frame with filtered zip codes

zc_filtered = GA_zc_t50[(GA_zc_t50['yr_avg']<q_60) & (GA_zc_t50['yr_avg']>q_35)]
print(f'Number of Zipcodes: {len(zc_filtered)}')

Number of Zipcodes: 7


In [23]:
zc_filtered['ROI'] = (zc_filtered['2018-04']/zc_filtered['1996-04'])-1

/Users/bryanjamieson/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
zc_filtered['std'] = zc_filtered.loc[:,'1996-04':'2018-04'].std(skipna=True, axis=1)

/Users/bryanjamieson/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
zc_filtered['mean'] = zc_filtered.loc[:,'1996-04':'2018-04'].mean(skipna=True, axis=1)

/Users/bryanjamieson/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [26]:
zc_filtered['CV'] = zc_filtered['std']/zc_filtered['mean']

/Users/bryanjamieson/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [27]:
zc_filtered[['RegionName','std','mean','ROI','CV']].head()

,RegionName,std,mean,ROI,CV
213,30080,39919.584647,213763.396226,1.338346,0.186747
221,30062,42293.095620,226361.509434,1.191333,0.186839
440,30066,28359.729404,176089.811321,1.028869,0.161053
495,30101,22891.755053,172782.264151,0.704562,0.132489
1017,30064,32823.525473,207401.132075,0.949728,0.158261


In [28]:
print(zc_filtered.CV.describe())
cv_upper = zc_filtered.CV.quantile(.7)
print(f'\nCV Upper Limit: {cv_upper}')

count    7.000000
mean     0.186609
std      0.069124
min      0.132489
25%      0.151275
50%      0.161053
75%      0.186793
max      0.336582
Name: CV, dtype: float64

CV Upper Limit: 0.18676501434287818


In [29]:
zc_best = zc_filtered[zc_filtered['CV']<cv_upper].sort_values('ROI', axis=0,ascending=False)[:5]

In [30]:
print('\n Top 5 Zipcodes to Invest in')
zc_best[['RegionName','ROI','CV']]


 Top 5 Zipcodes to Invest in


,RegionName,ROI,CV
213,30080,1.338346,0.186747
440,30066,1.028869,0.161053
1017,30064,0.949728,0.158261
1797,30152,0.818302,0.144289
495,30101,0.704562,0.132489


In [31]:
# 1) 30080
# 2) 30066
# 3) 30064
# 4) 30152
# 5) 30101

In [32]:
#for i in range(5):
#    zc_best[i].value.plot(label=zc_best[i].RegionName[0],figsize=(15,6))
#    plt.legend()

# Step 2: Data Preprocessing

In [33]:
# Get data in propper formatting.
# dates, values
# region name as name, dates as index column

In [34]:
def get_datetimes(df,start_col=1):
    return pd.to_datetime(df.columns.values[start_col:], format='%Y-%m')

In [35]:
new_df  = pd.DataFrame()
new_df['dates'] = get_datetimes(df,7)

In [36]:
#dates = df_GA_2.columns[7:]
#print(dates)

# Format data from horizontal to vertical
* Grab the date values from each zip code and make it a column in our new dataframe

In [37]:
#new_df  = pd.DataFrame()
#new_df['dates'] = dates

#for index in range(df_GA_2.shape[0]):
#    row = df_GA_2.iloc[index]
#    region_name = row['RegionName']
#    date_values = row[dates].values
#    new_df[region_name] = date_values
#new_df.head()

In [38]:
#for zipcode in df.RegionName:
#    row = df[df.RegionName==zipcode].iloc[:,7:]
#    new_df[zipcode] = row.values[0]
#new_df.set_index('dates',inplace=True)

In [39]:
#print(new_df.shape)
#new_df.head()

In [40]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 1 columns):
dates    265 non-null datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 2.2 KB


# Change 'dates' column to datetime format

In [41]:
#Did this above

# Set 'dates' column as our index

In [42]:
# Create New DF containing summary time series for region and each of the 3 counties.

In [43]:
Fulton_zipcodes = list(df_GA_2[df_GA_2.CountyName=='Fulton']['RegionName'])
Dekalb_zipcodes = list(df_GA_2[df_GA_2.CountyName=='Dekalb']['RegionName'])
Cobb_zipcodes = list(df_GA_2[df_GA_2.CountyName=='Cobb']['RegionName'])

In [44]:
print(Fulton_zipcodes)

[30331, 30022, 30004, 30075, 30309, 30076, 30324, 30350, 30328, 30316, 30305, 30213, 30342, 30308, 30005, 30306, 30327, 30307, 30291, 30009, 30317, 30326, 30268, 30363]


In [45]:
print(Dekalb_zipcodes)

[30058, 30319, 30032, 30034, 30038, 30084, 30338, 30294, 30033, 30087, 30329, 30341, 30030, 30360, 30288, 30002, 30079]


In [46]:
print(Cobb_zipcodes)

[30080, 30062, 30127, 30066, 30101, 30067, 30144, 30064, 30339, 30126, 30152, 30102, 30060, 30068, 30082, 30008, 30168, 30106]


In [47]:
summary_df = pd.DataFrame()
summary_df['Region'] = new_df.mean(axis=1)
summary_df['Fulton'] = new_df[Fulton_zipcodes].mean(axis=1)
summary_df['Dekalb'] = new_df[Dekalb_zipcodes].mean(axis=1)
summary_df['Cobb'] = new_df[Cobb_zipcodes].mean(axis=1)

KeyError: "None of [Int64Index([30331, 30022, 30004, 30075, 30309, 30076, 30324, 30350, 30328,\n            30316, 30305, 30213, 30342, 30308, 30005, 30306, 30327, 30307,\n            30291, 30009, 30317, 30326, 30268, 30363],\n           dtype='int64')] are in the [columns]"

# Step 3: EDA and Visualization

In [ ]:
import matplotlib
font = {
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

In [ ]:
#plt.gcf().autofmt_xdate()

In [ ]:
#new_df.plot(figsize=(20,7),legend=True)
#plt.title("Mean Real Estate Prices for zipcodes in Fulton, Dekalb & Cobb County",{'size':22});
#plt.xlabel("Year",{'size':22})
#plt.ylabel("$",{'size':22})

In [ ]:
summary_df.plot(figsize=(20,7),legend=True)
plt.title("Mean Real Estate Prices in Fulton, Dekalb and Cobb Counties",{'size':22});
plt.xlabel("Year",{'size':22})
plt.ylabel("$",{'size':22})

# change index

In [ ]:
# Above we can see that Fulton is the most expensive county to live in and it's mean price has increased the most

In [ ]:
# insert zip code selection from "https://medium.com/@feraguilari/time-series-analysis-modfinalproyect-b9fb23c28309"
# Find top 5 zipcodes in Fulton by ROI - aka add new cell with ROI by zip
# Then do forcast for 10 year ROI

In [ ]:
# Add Rolling mean and std for zipcodes
# Add ADFuller test

# Step 4: Reshape from Wide to Long Format

In [ ]:
# Time Series Analysis here

In [ ]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['RegionName', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

# Step 5: ARIMA Modeling

In [ ]:
split_date = '2017-08'
training, validation = summary_df[:split_date].Region, summary_df[split_date:].Region
ts = summary_df.Region

In [ ]:
p_values = [0, 1, 2, 4, 6, 8, 10]
d_values = range(0, 3)
q_values = range(0, 3)
warnings.filterwarnings("ignore")
evaluate_models(summary_df.values, p_values, d_values, q_values)

# Step 6: Interpreting Results